In [2]:
import pandas as pd

data = pd.read_csv('Eng_Kin-Paralleldata.csv')

print(data)

                ID                                            English  \
0      ID_IFVTMXW8        F_all back to threading messages by subject   
1      ID_SBXUPXE3  You will have to restart GIMP for the followin...   
2      ID_ZOCZ3PXQ  The radio button whose group this widget belon...   
3      ID_6AVBJ1YS           Print all subdirectories in a directory.   
4      ID_FW4V44WK  Paste the contents of the clipboard as a new i...   
...            ...                                                ...   
20585  ID_T3D2IRD5  Yes. We plan to support major CCD cameras and ...   
20586  ID_JWVP5ZDJ  Most likely you did not configure correctly (o...   
20587  ID_LI4BAED3  ... too numerous others to mention who also le...   
20588  ID_JMR2T84U  & kiten; can deinflect verbs you search for. T...   
20589  ID_3BG7KZ98  Next you will find a full overview on all comm...   

                                             Kinyarwanda  
0                     Inyuma Kuri Ubutumwa ku Ikivugwaho  
1    